# Implementacja DFT
## Laboratorium 10 - Metody Obliczeniowe w Nauce i Technice

In [36]:
import numpy as np
import cmath
import os
import matplotlib.pyplot as plt
import time

## Zadanie 1. DFT.

### 1.
Zaimplementuj funkcję realizującą DFT jako iloczyn macierzy Fouriera $\boldsymbol{F}_n$ i $n$-elementowego wektora wejściowego ($\boldsymbol{y} = \boldsymbol{F}_n\boldsymbol{x}$).
$$
    n = 2^r \\
    [\boldsymbol{F}_n]_{jk} = \xi^{jk} \\
    \xi = e^{-\frac{2\pi i}{n}} = \cos{\left(\frac{2\pi}{n}\right)} - i\sin{\left(\frac{2\pi}{n}\right)} = \overline{\omega}
$$

In [31]:
def xi(n):
    PI = np.pi
    return cmath.exp(-2j*PI / n)


def fill_fouriers_matrix(F, XI):
    n = len(F)
    for j in range(n):
        for k in range(n):
            F[j][k] = XI**(j * k)
            
    return F


def dft(x, n):
    XI = xi(n)
    F = np.empty((n, n), dtype=np.csingle)
    F = fill_fouriers_matrix(F, XI)
    
    y = F @ x
    return y, F       


n = 2**3
x = np.random.randint(0, 101, n)
y, F = dft(x, n)
y

array([ 364.         +0.j       ,  -82.99137753-29.8908723j,
         15.        -35.j       ,  -25.00862247-47.8908723j,
       -146.         +0.j       ,  -25.00862247+47.8908723j,
         15.        +35.j       ,  -82.99137753+29.8908723j])

### 2.
Zaimplementuj również IDFT korzystając z tożsamosci:
$$
    \boldsymbol{F}_n^{-1}\boldsymbol{y} = \frac{\overline{\boldsymbol{F}}_n\boldsymbol{y}}{n} = \frac{\overline{\boldsymbol{F}_n\overline{\boldsymbol{y}}}}{n}
$$
Sprawdź poprawność działania funkcji realizującej DFT stosując transformację odwrotną ($\boldsymbol{x} = \boldsymbol{F}_n^{−1}\boldsymbol{y}$) oraz porównując uzyskane wyniki z wyjściem funkcji bibliotecznej.

In [43]:
# Evaluating inverse x vector
x_idft = np.conjugate(F @ np.conjugate(y)) / n

# Comparing own DFT with own IDFT
print(f"x{os.linesep}", x, end=2 * os.linesep)
print(f"x given by IDFT{os.linesep}", x_idft, end=2 * os.linesep)
print("Comparing above 2 matrices: ", np.allclose(x, x_idft))

print(2 * os.linesep)

# Comparing own DFT with NumPy's FFT
print(f"y{os.linesep}", y, end=2 * os.linesep)
y_fft = np.fft.fft(x)
print(f"y given by NumPy's FFT{os.linesep}", y_fft, end=2 * os.linesep)
print("Comparing above 2 matrices: ", np.allclose(y, y_fft))

x
 [ 4 76 19 79 58 69 28 31]

x given by IDFT
 [ 4.        -0.j 75.99999988-0.j 19.        -0.j 78.99999918-0.j
 58.        -0.j 69.00000012-0.j 28.        -0.j 31.00000082-0.j]

Comparing above 2 matrices:  True



y
 [ 364.         +0.j         -82.99137753-29.8908723j
   15.        -35.j         -25.00862247-47.8908723j
 -146.         +0.j         -25.00862247+47.8908723j
   15.        +35.j         -82.99137753+29.8908723j]

y given by NumPy's FFT
 [ 364.         +0.j          -82.99137803-29.89087297j
   15.        -35.j          -25.00862197-47.89087297j
 -146.         +0.j          -25.00862197+47.89087297j
   15.        +35.j          -82.99137803+29.89087297j]

Comparing above 2 matrices:  True


### 3.
Zaimplementuj rekurencyjny algorytm Cooleya-Turkeya realizujący szybką transformację Fouriera (FFT). Porównaj szybkość jego działania z implementacją biblioteczną oraz implementacją z mnożeniem wektora przez macierz $\boldsymbol{F}_n$ dla danych
o różnym rozmiarze.

In [48]:
def fft(x):
    n = len(x)
    if n == 1:
        return x
    
    x0 = [x[i] for i in range(0, n, 2)]
    x1 = [x[i] for i in range(1, n, 2)]
    
    y0 = fft(x0)
    y1 = fft(x1)
    
    omega = cmath.exp(2j * np.pi * n)
    
    y = np.empty(n, dtype=np.csingle)
    for j in range(n // 2):
        y[j] = y0[j] + omega**j * y1[j]
        y[j + n // 2] = y0[j] + omega**(j + n // 2) * y0[j]
        
    return y


fft(x)

array([364.+0.0000000e+00j, 356.-8.8664427e-13j, 414.-1.5420753e-12j,
       320.-2.3356464e-12j, 218.-8.5431358e-13j,  92.-5.4766207e-13j,
       248.-1.7007895e-12j,  32.-2.7432088e-13j], dtype=complex64)

In [52]:
# Comparing own FFT with NumPy's FFT
# y_fft = fft(x)
# print(f"y{os.linesep}", y_fft, end=2 * os.linesep)
# y_npfft = np.fft.fft(x)
# print(f"y given by NumPy's FFT{os.linesep}", y_npfft, end=2 * os.linesep)
# print("Comparing above 2 matrices: ", np.allclose(y_fft, y_npfft))

#### Porównanie szybkości algorytmów

In [55]:
for r in range(13):
    print(f"Performance comparison for n = 2^{r}")
    n = 2**r
    x = np.random.randint(0, 101, n)
    
    print("Own DFT... ", end='')
    start = time.time()
    y, F = dft(x, n)
    end = time.time()
    print(f"{end - start} s")
    
    print("Own FFT... ", end='')
    start = time.time()
    y = fft(x)
    end = time.time()
    print(f"{end - start} s")
    
    print("NumPy's FFT... ", end='')
    start = time.time()
    y = np.fft.fft(x)
    end = time.time()
    print(f"{end - start} s")
    
    print(2 * os.linesep)

Performance comparison for n = 2^0
Own DFT... 0.0019910335540771484 s
Own FFT... 0.0 s
NumPy's FFT... 0.0 s



Performance comparison for n = 2^1
Own DFT... 0.0 s
Own FFT... 0.0 s
NumPy's FFT... 0.001008749008178711 s



Performance comparison for n = 2^2
Own DFT... 0.0 s
Own FFT... 0.0 s
NumPy's FFT... 0.0 s



Performance comparison for n = 2^3
Own DFT... 0.0 s
Own FFT... 0.0 s
NumPy's FFT... 0.0009970664978027344 s



Performance comparison for n = 2^4
Own DFT... 0.0 s
Own FFT... 0.0 s
NumPy's FFT... 0.0 s



Performance comparison for n = 2^5
Own DFT... 0.0 s
Own FFT... 0.0010135173797607422 s
NumPy's FFT... 0.0 s



Performance comparison for n = 2^6
Own DFT... 0.003568410873413086 s
Own FFT... 0.003039121627807617 s
NumPy's FFT... 0.0 s



Performance comparison for n = 2^7
Own DFT... 0.010551691055297852 s
Own FFT... 0.007554531097412109 s
NumPy's FFT... 0.0 s



Performance comparison for n = 2^8
Own DFT... 0.04467344284057617 s
Own FFT... 0.016399383544921875 s
NumPy's FFT... 

## Zadanie 2. DFT w 1D.

In [ ]:
n = 5
S = np.empty(n, dtype=np.ndarray)
for i, t in enumerate(((.5, 1), (1, 1), (1, .5), (3, .25), (2, 2.5))):
    a, b = t
    S[i] = 